In [2]:
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
import warnings

warnings.filterwarnings('ignore', category = FutureWarning)

In [5]:
path_scrapper = "/content/drive/MyDrive/ProjetML/Base_BookCrossing/Base_completee/"
file_scrapper = path_scrapper         + "bothWebSites_InternetSearch_AllBooks_BookCrossing.csv"
file_scrapper_cleaned = path_scrapper + "bothWebSites_InternetSearch_AllBooks_BookCrossing_cleaned.csv"
file_scrapper_w2v = path_scrapper     + "bothWebSites_InternetSearch_AllBooks_BookCrossing_cleaned_w2v.csv"

In [ ]:
books_scrapped = pd.read_csv(file_scrapper, sep = ';', parse_dates=['Year-Of-Publication'], dtype = {"ISBN_13": str})

len(books_scrapped)

114528

In [ ]:
books_scrapped.ISBN_13[0]

'9,78E+12'

In [ ]:
books_scrapped_cleaned = books_scrapped.copy()

# Transformation of column names to be as our SQL data base
Columns :
    - 'book_id'
    - 'average_rating'
    - 'issue_flag'
    - 'is_active' 
    - 'data_source' 
do not exist in books_scrapped

In [ ]:
books_scrapped_cleaned.rename( columns = {'ISBN_10': 'isbn', 'ISBN_13': 'isbn_13', 'Book-Title': 'book_title', \
                                  'Book-Author': 'book_author', 'Year-Of-Publication': 'year_of_publication', \
                                  'Description': 'book_description', 'Language': 'book_language', \
                                  'Pages': 'number_of_pages', 'Awards': 'awards', 'Author\'s genre': 'author_genres', \
                                  'Same serie': 'books_in_series'}, inplace = True )

In [ ]:
books_scrapped_cleaned.to_csv(file_scrapper_cleaned, sep = "|", index = False)

# Creation of a second column for the second author gender

In [ ]:
books_scrapped_cleaned = pd.read_csv(file_scrapper_cleaned, sep = '|')

In [ ]:
books_scrapped_cleaned.head(20)

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,book_description,book_language,Image,number_of_pages,awards,author_genres,books_in_series
0,195153448,"9,78E+12",0.0,Classical Mythology,Mark P. O. Morford,2003-01-01,"Oxford University Press, USA",Social Science,Provides an introduction to classical myths pl...,en,http://books.google.com/books/content?id=YABYK...,808.0,NaN,NaN,NaN
1,2005018,"9,78E+12",0.0,Clara Callan : A Novel,Richard Bruce Wright,2001-01-01,HarperFlamingo Canada,Actresses,"In a small town in Canada, Clara Callan reluct...",en,http://books.google.com/books/content?id=yfx0v...,414.0,"\nScotiabank Giller Prize (2001), Governor Gen...",NaN,[]
2,60973129,0,IND:30000026059836,Decision in Normandy,Carlo D'Este,1991-01-01,Harper Perennial,1940-1949,"Here, for the first time in paperback, is an o...",en,http://books.google.com/books/content?id=_LufA...,555.0,NaN,History,[]
3,374157065,"9,78E+12",0,Flu : The Story of the Great Influenza Pandemi...,Gina Bari Kolata,1999-01-01,Macmillan,Medical,"Describes the great flu epidemic of 1918, an o...",en,http://books.google.com/books/content?id=GkthX...,330.0,NaN,Health,[]
4,393045218,"9,78E+12",0.0,The Mummies of Ürümchi,E. J. W. Barber,1999-01-01,W. W. Norton,Design,A look at the incredibly well-preserved ancien...,en,http://books.google.com/books/content?id=5OujQ...,240.0,\nKiriyama Prize Nominee for Nonfiction (1999)\n,History,[]
5,399135782,"9,78E+12",0.0,The Kitchen God's Wife,Amy Tan,1991-01-01,Putnam Publishing Group,Fiction,A Chinese immigrant who is convinced she is dy...,en,http://books.google.com/books/content?id=s6IRK...,415.0,NaN,Literature & Fiction,[]
6,425176428,"9,78E+12",0.0,What If? : The World's Foremost Military Histo...,Robert Cowley,2000-01-01,Berkley Publishing Group,History,"Essays by respected military historians, inclu...",en,http://books.google.com/books/content?id=LxevD...,395.0,NaN,History,"['https://www.goodreads.com/book/show/406281',..."
7,671870432,"9,78E+12",0.0,Pleading Guilty,Scott Turow,1993-01-01,Simon & Schuster Audio,Fiction,NaN,en,NaN,0.0,NaN,Mystery & Thrillers,"['https://www.goodreads.com/book/show/425029',..."
8,679425608,0,UVA:35007001566771,Under the Black Flag : The Romance and the Rea...,David Cordingly,1995-01-01,Random House Incorporated,Fiction,"An authoritative, revisionist history of the g...",en,http://books.google.com/books/content?id=slQSA...,296.0,NaN,History,[]
9,074322678X,0.0,PSU:000049152678,Where You'll Find Me : And Other Stories,Ann Beattie,2002-01-01,Scribner,Fiction,"Now back in print, Ann Beattie&#39;s finest sh...",un,http://books.google.com/books/content?id=twdaA...,207.0,NaN,Literature & Fiction,[]


In [ ]:
#new column created
books_scrapped_cleaned.insert(14, "author_genres_other", pd.Series())

#We only deal with not empty cells
aut_gen = books_scrapped_cleaned.author_genres[books_scrapped_cleaned["author_genres"].isna() == False]

for i in aut_gen.index:
    gender = books_scrapped_cleaned["author_genres"][i].strip()

    res_split = gender.strip().split('&')

    #In case a split has been done:
    ##### the second author gender must be copied in author_genres_other column
    ##### and the first author gender only must appear in author_genres column
    if(len(res_split) > 1):
        books_scrapped_cleaned["author_genres_other"][i] = res_split[1].strip()
        books_scrapped_cleaned["author_genres"][i] = res_split[0].strip()       

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value i

In [ ]:
books_scrapped_cleaned.head(20)

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,book_description,book_language,Image,number_of_pages,awards,author_genres,author_genres_other,books_in_series
0,195153448,"9,78E+12",0.0,Classical Mythology,Mark P. O. Morford,2003-01-01,"Oxford University Press, USA",Social Science,Provides an introduction to classical myths pl...,en,http://books.google.com/books/content?id=YABYK...,808.0,NaN,NaN,NaN,NaN
1,2005018,"9,78E+12",0.0,Clara Callan : A Novel,Richard Bruce Wright,2001-01-01,HarperFlamingo Canada,Actresses,"In a small town in Canada, Clara Callan reluct...",en,http://books.google.com/books/content?id=yfx0v...,414.0,"\nScotiabank Giller Prize (2001), Governor Gen...",NaN,NaN,[]
2,60973129,0,IND:30000026059836,Decision in Normandy,Carlo D'Este,1991-01-01,Harper Perennial,1940-1949,"Here, for the first time in paperback, is an o...",en,http://books.google.com/books/content?id=_LufA...,555.0,NaN,History,NaN,[]
3,374157065,"9,78E+12",0,Flu : The Story of the Great Influenza Pandemi...,Gina Bari Kolata,1999-01-01,Macmillan,Medical,"Describes the great flu epidemic of 1918, an o...",en,http://books.google.com/books/content?id=GkthX...,330.0,NaN,Health,NaN,[]
4,393045218,"9,78E+12",0.0,The Mummies of Ürümchi,E. J. W. Barber,1999-01-01,W. W. Norton,Design,A look at the incredibly well-preserved ancien...,en,http://books.google.com/books/content?id=5OujQ...,240.0,\nKiriyama Prize Nominee for Nonfiction (1999)\n,History,NaN,[]
5,399135782,"9,78E+12",0.0,The Kitchen God's Wife,Amy Tan,1991-01-01,Putnam Publishing Group,Fiction,A Chinese immigrant who is convinced she is dy...,en,http://books.google.com/books/content?id=s6IRK...,415.0,NaN,Literature,Fiction,[]
6,425176428,"9,78E+12",0.0,What If? : The World's Foremost Military Histo...,Robert Cowley,2000-01-01,Berkley Publishing Group,History,"Essays by respected military historians, inclu...",en,http://books.google.com/books/content?id=LxevD...,395.0,NaN,History,NaN,"['https://www.goodreads.com/book/show/406281',..."
7,671870432,"9,78E+12",0.0,Pleading Guilty,Scott Turow,1993-01-01,Simon & Schuster Audio,Fiction,NaN,en,NaN,0.0,NaN,Mystery,Thrillers,"['https://www.goodreads.com/book/show/425029',..."
8,679425608,0,UVA:35007001566771,Under the Black Flag : The Romance and the Rea...,David Cordingly,1995-01-01,Random House Incorporated,Fiction,"An authoritative, revisionist history of the g...",en,http://books.google.com/books/content?id=slQSA...,296.0,NaN,History,NaN,[]
9,074322678X,0.0,PSU:000049152678,Where You'll Find Me : And Other Stories,Ann Beattie,2002-01-01,Scribner,Fiction,"Now back in print, Ann Beattie&#39;s finest sh...",un,http://books.google.com/books/content?id=twdaA...,207.0,NaN,Literature,Fiction,[]


In [ ]:
books_scrapped_cleaned.to_csv(file_scrapper_cleaned, sep = "|", index = False)

# Cleaning of language column

##### First some not conform language are manually transformed

In [ ]:
books_scrapped_cleaned = pd.read_csv(file_scrapper_cleaned, sep = '|')

In [ ]:
books_scrapped_cleaned.book_language[books_scrapped_cleaned.book_language == "Catalan; Valencian"] = "ca"

In [ ]:
books_scrapped_cleaned.book_language[books_scrapped_cleaned.book_language == "English, Middle (1100-1500)"] = "en"

In [ ]:
books_scrapped_cleaned.book_language[books_scrapped_cleaned.book_language == "Multiple languages"] = "en"

In [ ]:
books_scrapped_cleaned.book_language[books_scrapped_cleaned.book_language == "zh-CN"] = "zh"

In [ ]:
books_scrapped_cleaned.book_language[books_scrapped_cleaned.book_language == "zh-TW"] = "zh"

##### Then automatic transformation is performed

In [ ]:
!pip install iso-639

In [ ]:
from iso639 import languages

In [ ]:
#When more than two letters to code the language, 
#then perform a transformation into ISO 639 coding 
lines_to_transf = books_scrapped_cleaned.book_language[books_scrapped_cleaned.book_language.str.len() > 2]
lines_to_transf.head()

In [ ]:
for i in lines_to_transf.index:
    books_scrapped_cleaned.book_language[i] = languages.get(name = books_scrapped_cleaned.book_language[i]).alpha2 

In [ ]:
books_scrapped_cleaned.to_csv(file_scrapper_cleaned, sep = "|", index = False)

# Cleaning of awards column

In [ ]:
books_scrapped_cleaned = pd.read_csv(file_scrapper_cleaned, sep = '|')

## Analysis of awards

In [ ]:
#We only deal with not empty cells
awards_lines = books_scrapped_cleaned.author_genres[books_scrapped_cleaned["awards"].isna() == False]

#study of the number of awards inside each cell
#study of the kind of awards
nb_awards = []
awards_name = []

for i in awards_lines.index:

    #first we retrieve the "\n", " " ...
    awards_cleaned = books_scrapped_cleaned["awards"][i].strip()

    #then we split when there are several awards in the same cell
    awards_split = awards_cleaned.split(',')
    l = len(awards_split)

    #27 awards for those books !
    if l == 27:
      print("\n\ntitle {0}, at indice {1}\n".format(books_scrapped_cleaned["book_title"][i], i))
      
    nb_awards.append(l)

    #In case a split has been done:
    if(l > 1):
        for j in range(l):
          #we keep only the name, not the year of obtention
          temp = awards_split[j].strip()
          awards_name.append(temp.split("(")[0].strip())
      



title Harry Potter and the Sorcerer's Stone, at indice 2143



title Harry Potter and the Sorcerer's Stone, at indice 2809



title Harry Potter and the Sorcerer's Stone, at indice 9027



title Harry Potter y la piedra filosofal, at indice 21110



title Harry Potter and the Sorcerer's Stone, at indice 28428



title Harry Potter und der Stein der Weisen, at indice 34539



title Harrius Potter et Philosophi Lapis : (Harry Potter and the Philosopher's Stone), at indice 35740



title Harry Potter and the Philosopher's Stone, at indice 50436



title Harry Potter e la pietra filosofale : romanzo, at indice 51993



title Harry Potter and the Sorcerer's Stone, at indice 52581



title Harry Potter and the Sorcerer's Stone, at indice 55787



title Harry Potter à l'école des sorciers, at indice 64658



title Harry Potter and the Sorcerer's Stone, at indice 72188



title Harry Potter and the Sorcerer's Stone, at indice 77387



title Harry Potter and the Philosopher's Stone : Irish Ga

In [ ]:
books_scrapped_cleaned["awards"][2809]

"\nMythopoeic Fantasy Award for Children's Literature (2008), British Book Award for Children's Book of the Year (1998), Prijs van de Nederlandse Kinderjury for 6-9 jaar en 10-12 jaar (2002), American Booksellers Book Of The Year  Award for Children (1999), Audie Award (2000)\n...more\nWest Australian Young Readers' Book Award (WAYRBA) for Younger Readers (2000), South Carolina Book Award for Junior Book Award (2001), Grand Canyon Reader Award for Teen Book (2000), Charlotte Award (2000), Nene Award (2000), Massachusetts Children's Book Award (2000), Colorado Blue Spruce Young Adult Book Award (2001), Blue Hen Book Award for Chapter Book (2001), Nevada Young Readers' Award for Young Reader Category (2000), Golden Archer Award for Middle/Junior High (2000), Indian Paintbrush Book Award (2000), Hotze de Roosprijs (2002), Nestlé Smarties Book Prize for 9–11 years (1997), Eliot Rosewater Indiana High School Book Award (2001), Kinderboekwinkelprijs (1999), Parenting Book of the Year Award (

In [ ]:
pd.Series(nb_awards).value_counts()

1     8858
2     3077
3     1249
4      658
5      641
6      143
7       83
8       69
10      28
12      27
9       19
15      16
21      16
27      16
11      11
16       7
14       5
20       4
19       2
13       2
17       1
dtype: int64

In [ ]:
max(nb_awards)

27

In [ ]:
stats_awards = pd.Series(awards_name).value_counts()
stats_awards

Dorothy Canfield Fisher Children's Book Award Nominee                       428
National Book Award Finalist for Fiction                                    233
National Book Critics Circle Award Nominee for Fiction                      224
Hugo Award Nominee for Best Novel                                           210
Newbery Medal Nominee                                                       202
                                                                           ... 
Wilhelm-Hauff-Preis                                                           1
Lambda Literary Award for Gay Men's Science Fiction/Fantasy                   1
Seiun Award 星雲賞 for Best Translated Short Story for "Story of Your Life"      1
Analog Award Nominee for Best Serial Novel or Novella for "Anasazi"           1
Gordon Montador Award                                                         1
Length: 2033, dtype: int64

In [ ]:
stats_awards[stats_awards.values > 100]

Dorothy Canfield Fisher Children's Book Award Nominee     428
National Book Award Finalist for Fiction                  233
National Book Critics Circle Award Nominee for Fiction    224
Hugo Award Nominee for Best Novel                         210
Newbery Medal Nominee                                     202
Newbery Medal                                             173
Nebula Award Nominee for Best Novel                       173
Los Angeles Times Book Prize Nominee for Fiction          171
Margaret A. Edwards Award                                 166
Hugo Award for Best Novel                                 161
Locus Award Nominee for Best Fantasy Novel                159
World Fantasy Award Nominee for Best Novel                156
Booker Prize Nominee                                      154
Lewis Carroll Shelf Award                                 144
Books I Loved Best Yearly                                 144
Pulitzer Prize for Fiction                                142
Internat

In [ ]:
type(stats_awards)

pandas.core.series.Series

In [ ]:
stats_awards.to_csv(path_scrapper + "awards_names.csv", sep = "|", index = True)

## Cleaning of awards

In [ ]:
stats_awards[0]

428

In [ ]:
stats_awards.index[0]

"Dorothy Canfield Fisher Children's Book Award Nominee"

In [ ]:
def searchInFamousAwards(list_awards):
  print("\n\n")

  l_entry = len(list_awards)
  l_ref = len(stats_awards)
  awards_returned = ""

  i = 0
  #among all awards existing, we search for the most famous appearing in list_awards
  while((awards_returned == "") and (i < l_ref)):
    if stats_awards.index[i] in list_awards:
      awards_returned = stats_awards.index[i] 
    i = i + 1  

  if(awards_returned == ""):
    awards_returned = list_awards[0]

  return  awards_returned


In [ ]:
books_scrapped_cleaned = pd.read_csv(file_scrapper_cleaned, sep = '|')

In [ ]:
#new column created
books_scrapped_cleaned["awards_cleaned"] = pd.Series()

#We only deal with not empty cells
awards_lines = books_scrapped_cleaned.author_genres[books_scrapped_cleaned["awards"].isna() == False]

for i in awards_lines.index:
    awards_cleaned = []

    #first we remove the "\n", " " ...
    awards_cleaned = books_scrapped_cleaned["awards"][i].strip()

    #then we split when there are several awards in the same cell
    awards_split = awards_cleaned.split(',')
    l = len(awards_split)

    #first we clean all the awards (because split introduce " ")
    for j in range(l):
      temp = awards_split[j].strip()
      awards_split[j] = temp.split("(")[0].strip()

    #In case a split has been done:
    if(l > 1):
        res_awards = searchInFamousAwards(awards_split)
        if res_awards != "":
          books_scrapped_cleaned["awards_cleaned"][i] = res_awards
        else:
          books_scrapped_cleaned["awards_cleaned"][i] = awards_split[0]



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.




























































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [ ]:
books_scrapped_cleaned["awards_cleaned"]

0                         NaN
1         Trillium Book Award
2                         NaN
3                         NaN
4                         NaN
                 ...         
114523                    NaN
114524                    NaN
114525                    NaN
114526                    NaN
114527                    NaN
Name: awards_cleaned, Length: 114528, dtype: object

In [ ]:
books_scrapped_cleaned["awards_cleaned"].value_counts()

Dorothy Canfield Fisher Children's Book Award Nominee                428
National Book Award Finalist for Fiction                             233
Hugo Award Nominee for Best Novel                                    210
National Book Critics Circle Award Nominee for Fiction               156
Books I Loved Best Yearly                                            138
                                                                    ... 
Analog Award Nominee for Best Novella/Novelette for "The Locusts"      1
Prijs van de Nederlandse Kinderjury for 6-9 jaar                       1
Will Eisner Comic Industry Awards for Best Comics-Related Book         1
Compton Crook Award                                                    1
O. Henry Award                                                         1
Name: awards_cleaned, Length: 613, dtype: int64

In [ ]:
del books_scrapped_cleaned["awards"]

In [ ]:
books_scrapped_cleaned.rename(columns = {"awards_cleaned": "awards"}, inplace = True)

In [ ]:
books_scrapped_cleaned.head()

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,awards
0,195153448,"9,78E+12",0.0,Classical Mythology,Mark P. O. Morford,2003-01-01,"Oxford University Press, USA",Social Science,Provides an introduction to classical myths pl...,en,http://books.google.com/books/content?id=YABYK...,808.0,NaN,NaN,NaN,NaN
1,2005018,"9,78E+12",0.0,Clara Callan : A Novel,Richard Bruce Wright,2001-01-01,HarperFlamingo Canada,Actresses,"In a small town in Canada, Clara Callan reluct...",en,http://books.google.com/books/content?id=yfx0v...,414.0,NaN,NaN,[],Trillium Book Award
2,60973129,0,IND:30000026059836,Decision in Normandy,Carlo D'Este,1991-01-01,Harper Perennial,1940-1949,"Here, for the first time in paperback, is an o...",en,http://books.google.com/books/content?id=_LufA...,555.0,History,NaN,[],NaN
3,374157065,"9,78E+12",0,Flu : The Story of the Great Influenza Pandemi...,Gina Bari Kolata,1999-01-01,Macmillan,Medical,"Describes the great flu epidemic of 1918, an o...",en,http://books.google.com/books/content?id=GkthX...,330.0,Health,NaN,[],NaN
4,393045218,"9,78E+12",0.0,The Mummies of Ürümchi,E. J. W. Barber,1999-01-01,W. W. Norton,Design,A look at the incredibly well-preserved ancien...,en,http://books.google.com/books/content?id=5OujQ...,240.0,History,NaN,[],NaN


In [ ]:
books_scrapped_cleaned.to_csv(file_scrapper_cleaned, sep = "|", index = False)

# Création of a second column for Category

## First we separate categories with "&" in two columns
The column "Category_other" has been created to take the second part of such categories (part after "&")

In [ ]:
books_scrapped_cleaned = pd.read_csv(file_scrapper_cleaned, sep = '|')

In [ ]:
books_scrapped_cleaned.head()

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,awards
0,195153448,"9,78E+12",0.0,Classical Mythology,Mark P. O. Morford,2003-01-01,"Oxford University Press, USA",Social Science,Provides an introduction to classical myths pl...,en,http://books.google.com/books/content?id=YABYK...,808.0,NaN,NaN,NaN,NaN
1,2005018,"9,78E+12",0.0,Clara Callan : A Novel,Richard Bruce Wright,2001-01-01,HarperFlamingo Canada,Actresses,"In a small town in Canada, Clara Callan reluct...",en,http://books.google.com/books/content?id=yfx0v...,414.0,NaN,NaN,[],Trillium Book Award
2,60973129,0,IND:30000026059836,Decision in Normandy,Carlo D'Este,1991-01-01,Harper Perennial,1940-1949,"Here, for the first time in paperback, is an o...",en,http://books.google.com/books/content?id=_LufA...,555.0,History,NaN,[],NaN
3,374157065,"9,78E+12",0,Flu : The Story of the Great Influenza Pandemi...,Gina Bari Kolata,1999-01-01,Macmillan,Medical,"Describes the great flu epidemic of 1918, an o...",en,http://books.google.com/books/content?id=GkthX...,330.0,Health,NaN,[],NaN
4,393045218,"9,78E+12",0.0,The Mummies of Ürümchi,E. J. W. Barber,1999-01-01,W. W. Norton,Design,A look at the incredibly well-preserved ancien...,en,http://books.google.com/books/content?id=5OujQ...,240.0,History,NaN,[],NaN


In [ ]:
#new column created
books_scrapped_cleaned.insert(8, "Category_other", pd.Series())

#We only deal with not empty cells
cate_gen = books_scrapped_cleaned.author_genres[books_scrapped_cleaned["Category"].isna() == False]

for i in cate_gen.index:
    cate = books_scrapped_cleaned["Category"][i].strip()
    res_split = cate.split('&')

    #In case a split has been done:
    ##### the second category must be copied in Category_other column
    ##### and the first category only must appear in Category column
    if(len(res_split) > 1):
        books_scrapped_cleaned["Category_other"][i] = res_split[1].strip()
        books_scrapped_cleaned["Category"][i] = res_split[0].strip()       

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarnin

In [ ]:
books_scrapped_cleaned.head()

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,awards
0,195153448,"9,78E+12",0.0,Classical Mythology,Mark P. O. Morford,2003-01-01,"Oxford University Press, USA",Social Science,NaN,Provides an introduction to classical myths pl...,en,http://books.google.com/books/content?id=YABYK...,808.0,NaN,NaN,NaN,NaN
1,2005018,"9,78E+12",0.0,Clara Callan : A Novel,Richard Bruce Wright,2001-01-01,HarperFlamingo Canada,Actresses,NaN,"In a small town in Canada, Clara Callan reluct...",en,http://books.google.com/books/content?id=yfx0v...,414.0,NaN,NaN,[],Trillium Book Award
2,60973129,0,IND:30000026059836,Decision in Normandy,Carlo D'Este,1991-01-01,Harper Perennial,1940-1949,NaN,"Here, for the first time in paperback, is an o...",en,http://books.google.com/books/content?id=_LufA...,555.0,History,NaN,[],NaN
3,374157065,"9,78E+12",0,Flu : The Story of the Great Influenza Pandemi...,Gina Bari Kolata,1999-01-01,Macmillan,Medical,NaN,"Describes the great flu epidemic of 1918, an o...",en,http://books.google.com/books/content?id=GkthX...,330.0,Health,NaN,[],NaN
4,393045218,"9,78E+12",0.0,The Mummies of Ürümchi,E. J. W. Barber,1999-01-01,W. W. Norton,Design,NaN,A look at the incredibly well-preserved ancien...,en,http://books.google.com/books/content?id=5OujQ...,240.0,History,NaN,[],NaN


In [ ]:
 books_scrapped_cleaned["Category_other"].value_counts()

Autobiography               3072
Economics                    983
Spirit                       905
Relationships                880
Fitness                      843
Disciplines                  372
Recreation                   349
Graphic Novels               317
Hobbies                      293
Home                         150
Engineering                  123
Activities                   114
Collectibles                  74
AUTOBIOGRAPHY                  9
GRAPHIC NOVELS.                3
RELATIONSHIPS                  3
ECONOMICS                      2
Answers                        1
SPIRIT                         1
FITNESS                        1
mystery stories, English       1
GRAPHIC NOVELS                 1
Name: Category_other, dtype: int64

In [ ]:
 books_scrapped_cleaned["Category"].head(150)

0          Social Science
1               Actresses
2               1940-1949
3                 Medical
4                  Design
              ...        
145               Fiction
146            Philosophy
147    Literary Criticism
148               Fiction
149              Classics
Name: Category, Length: 150, dtype: object

## The same for categories with "(...)" (in Category column)
The part between "(...)" has been put in the second column

In [ ]:
#We only deal with not empty cells
cate_gen = books_scrapped_cleaned.Category[books_scrapped_cleaned["Category"].isna() == False]

for i in cate_gen.index:
    cate = books_scrapped_cleaned["Category"][i].strip()
    res_split_ent = cate.split('(')

    if (len(res_split_ent) > 1):
      res_split_out = res_split_ent[1].split(')')

      #In case a split has been done:
      ##### the second term (between "(...)") must be copied in Category_other column
      ##### and the first term remains in Category column (without the second term)
      books_scrapped_cleaned["Category"][i] = res_split_ent[0].strip()
      books_scrapped_cleaned["Category_other"][i] = res_split_out[0].strip()
    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [ ]:
books_scrapped_cleaned["Category"].head(50)

0       Social Science
1            Actresses
2            1940-1949
3              Medical
4               Design
5              Fiction
6              History
7              Fiction
8              Fiction
9              Fiction
10             Fiction
11             History
12             Fiction
13                 NaN
14          Nonfiction
15             Mystery
16              Nature
17             Fiction
18             Fiction
19             Fiction
20               Humor
21             Cooking
22           Reference
23    Juvenile Fiction
24    Juvenile Fiction
25    Juvenile Fiction
26             Fiction
27             Fiction
28             Fiction
29             Fiction
30             Fiction
31             Fiction
32    Canadian fiction
33                 NaN
34          Historical
35                 NaN
36             Fiction
37             Fiction
38             History
39             Fiction
40             Fiction
41            Classics
42              Health
43         

## Then removing of "," 
THe second term is, as previously, put inside "Category_other" column

In [ ]:
#We only deal with not empty cells
cate_gen = books_scrapped_cleaned.author_genres[books_scrapped_cleaned["Category"].isna() == False]

for i in cate_gen.index:
    cate = books_scrapped_cleaned["Category"][i].strip()
    res_split = cate.split(',')
    second_term = ""

    if (len(res_split) > 1):
      #In case a split has been done:
      ##### and the first term (before ",") remains in Category column (without the second term)
      books_scrapped_cleaned["Category"][i] = res_split[0].strip()

      ##### all the terms appearing after "," must be copied in Category_other column
      for j in range(1,len(res_split)):
        second_term = second_term + res_split[j]
      books_scrapped_cleaned["Category_other"][i] = second_term.strip()

    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
books_scrapped_cleaned["Category"].value_counts()

Fiction                    40402
Juvenile Fiction            5893
Biography                   3341
Mystery                     2894
History                     1998
                           ...  
Trans-Alaska Pipeline          1
Feral children                 1
Deutsch - Sprachkritik         1
Bereavement in children        1
Knots and splices              1
Name: Category, Length: 4579, dtype: int64

In [ ]:
books_scrapped_cleaned.to_csv(file_scrapper_cleaned, sep = "|", index = False)

# Stemming of columns
- Category and Category_other
- author_genres and author_genres_other

In [ ]:
books_scrapped_cleaned = pd.read_csv(file_scrapper_cleaned, sep = '|')

In [ ]:
books_scrapped_cleaned.head()

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,awards
0,195153448,"9,78E+12",0.0,Classical Mythology,Mark P. O. Morford,2003-01-01,"Oxford University Press, USA",Social Science,NaN,Provides an introduction to classical myths pl...,en,http://books.google.com/books/content?id=YABYK...,808.0,NaN,NaN,NaN,NaN
1,2005018,"9,78E+12",0.0,Clara Callan : A Novel,Richard Bruce Wright,2001-01-01,HarperFlamingo Canada,Actresses,NaN,"In a small town in Canada, Clara Callan reluct...",en,http://books.google.com/books/content?id=yfx0v...,414.0,NaN,NaN,[],Trillium Book Award
2,60973129,0,IND:30000026059836,Decision in Normandy,Carlo D'Este,1991-01-01,Harper Perennial,1940-1949,NaN,"Here, for the first time in paperback, is an o...",en,http://books.google.com/books/content?id=_LufA...,555.0,History,NaN,[],NaN
3,374157065,"9,78E+12",0,Flu : The Story of the Great Influenza Pandemi...,Gina Bari Kolata,1999-01-01,Macmillan,Medical,NaN,"Describes the great flu epidemic of 1918, an o...",en,http://books.google.com/books/content?id=GkthX...,330.0,Health,NaN,[],NaN
4,393045218,"9,78E+12",0.0,The Mummies of Ürümchi,E. J. W. Barber,1999-01-01,W. W. Norton,Design,NaN,A look at the incredibly well-preserved ancien...,en,http://books.google.com/books/content?id=5OujQ...,240.0,History,NaN,[],NaN


##### NLTK stemmer

In [ ]:
import nltk
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [ ]:
lines_OK = books_scrapped_cleaned.Category[books_scrapped_cleaned["Category"].isna() == False]

for i in lines_OK.index:
    w = books_scrapped_cleaned["Category"][i]
    books_scrapped_cleaned["Category"][i] = ps.stem(w)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
books_scrapped_cleaned.Category.head()

0    social sci
1       actress
2     1940-1949
3         medic
4        design
Name: Category, dtype: object

In [ ]:
lines_OK = books_scrapped_cleaned.Category_other[books_scrapped_cleaned["Category_other"].isna() == False]

for i in lines_OK.index:
    w = books_scrapped_cleaned["Category_other"][i]
    books_scrapped_cleaned["Category_other"][i] = ps.stem(w)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
lines_OK = books_scrapped_cleaned.author_genres[books_scrapped_cleaned["author_genres"].isna() == False]

for i in lines_OK.index:
    w = books_scrapped_cleaned["author_genres"][i]
    books_scrapped_cleaned["author_genres"][i] = ps.stem(w)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
lines_OK = books_scrapped_cleaned.author_genres_other[books_scrapped_cleaned["author_genres_other"].isna() == False]

for i in lines_OK.index:
    w = books_scrapped_cleaned["author_genres_other"][i]
    books_scrapped_cleaned["author_genres_other"][i] = ps.stem(w)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
books_scrapped_cleaned.to_csv(file_scrapper_cleaned, sep = "|", index = False)

# Removing books duplicates

##### Search by ISBN

In [ ]:
books_scrapped_cleaned = pd.read_csv(file_scrapper_cleaned, sep = '|')

In [ ]:
len(books_scrapped_cleaned.isbn.unique())

112831

In [ ]:
len(books_scrapped_cleaned)

114528

###### Tests drop

In [ ]:
df = pd.DataFrame(np.arange(32).reshape(-1, 4),

                  columns=['A', 'B', 'C', 'D'])

In [ ]:
df

,A,B,C,D
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19
5,20,21,22,23
6,24,25,26,27
7,28,29,30,31


In [ ]:
df = df.drop(2, axis = 0)
df

,A,B,C,D
0,0,1,2,3
1,4,5,6,7
3,12,13,14,15
4,16,17,18,19
5,20,21,22,23
6,24,25,26,27
7,28,29,30,31


In [ ]:
df.drop(2, axis = 0)

KeyError: ignored

In [ ]:
df.drop(3, axis = 0)

,A,B,C,D
0,0,1,2,3
1,4,5,6,7
4,16,17,18,19
5,20,21,22,23
6,24,25,26,27
7,28,29,30,31


In [ ]:
df.A[5]

20

In [ ]:
df.loc[6]

A    24
B    25
C    26
D    27
Name: 6, dtype: int64

###### True cleaning

In [ ]:
#for about 114 000 books, execution in 4 min

lines_OK = books_scrapped_cleaned.isbn[books_scrapped_cleaned.isbn.isna() == False]

#we construct as we go along the list of unique ISBN
listISBN = []

for i in lines_OK.index:
  #in case where this ISBN has already been found (so in listISBN)
  #we have the remove the corresponding line
  if (books_scrapped_cleaned.isbn[i] in listISBN):
    books_scrapped_cleaned.drop(i, axis = 0, inplace = True)
  else:
    listISBN.append(books_scrapped_cleaned.isbn[i])

#when a line is removed, its index also is removed
#so this function renumbers the indexes from 0 to n
books_scrapped_cleaned.reset_index(inplace = True, drop = True)

In [ ]:
len(books_scrapped_cleaned)

112833

In [ ]:
books_scrapped_cleaned.head(1)

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,awards
0,195153448,"9,78E+12",0.0,Classical Mythology,Mark P. O. Morford,2003-01-01,"Oxford University Press, USA",social sci,NaN,Provides an introduction to classical myths pl...,en,http://books.google.com/books/content?id=YABYK...,808.0,NaN,NaN,NaN,NaN


In [ ]:
books_scrapped_cleaned.to_csv(file_scrapper_cleaned, sep = "|", index = False)

##### Search by Title + Author

In [ ]:
books_scrapped_cleaned = pd.read_csv(file_scrapper_cleaned, sep = '|')

In [ ]:
print(len(books_scrapped_cleaned))
books_scrapped_cleaned.head(1)

112833


,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,awards
0,195153448,"9,78E+12",0.0,Classical Mythology,Mark P. O. Morford,2003-01-01,"Oxford University Press, USA",social sci,NaN,Provides an introduction to classical myths pl...,en,http://books.google.com/books/content?id=YABYK...,808.0,NaN,NaN,NaN,NaN


###### Tests NA lines

In [ ]:
Temp = books_scrapped_cleaned[["book_title", "book_author"]]
Temp.head(1)

,book_title,book_author
0,Classical Mythology,Mark P. O. Morford


In [ ]:
len(books_scrapped_cleaned.book_author[books_scrapped_cleaned.book_author.isna() == False])

111892

In [ ]:
Temp = Temp[Temp.book_title.isna() == False]
len(Temp)

112830

In [ ]:
Temp = Temp[Temp.book_author.isna() == False]
len(Temp)

111892

###### True cleaning

In [ ]:
#for about 114 000 books, execution in 13 min

lines_OK = books_scrapped_cleaned.book_author[books_scrapped_cleaned.book_author.isna() == False]
print(lines_OK.index)

#save of the duplicates
df_duplicates = pd.DataFrame(columns = books_scrapped_cleaned.columns)

#we construct as we go along the list of unique tuple (Title, Author)
listTitleAuthor = []
nb_duplicates = 0

for i in lines_OK.index:

  #in case where this couple (title, author) has already been found (so in listTitleAuthor)
  #we have the remove the corresponding line
  if ((books_scrapped_cleaned.book_title[i], books_scrapped_cleaned.book_author[i]) in listTitleAuthor):
    #lines removed are saved in another dataframe
    #print("\n{2} {0} {1}".format(books_scrapped_cleaned.book_title[i], books_scrapped_cleaned.book_author[i], i))
    df_duplicates.loc[nb_duplicates] = books_scrapped_cleaned.loc[i]
    nb_duplicates = nb_duplicates + 1

    books_scrapped_cleaned.drop(i, axis = 0, inplace = True)
  else:
    listTitleAuthor.append((books_scrapped_cleaned.book_title[i], books_scrapped_cleaned.book_author[i]))

#when a line is removed, its index also is removed
#so this function renumbers the indexes from 0 to n
books_scrapped_cleaned.reset_index(inplace = True, drop = True)

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            112823, 112824, 112825, 112826, 112827, 112828, 112829, 112830,
            112831, 112832],
           dtype='int64', length=111892)


In [48]:
print(len(df_duplicates))
df_duplicates.loc[12741]

13291


isbn                                                          1565111893
isbn_13                                                              0.0
OtherID                                                              0.0
book_title             100 Ways to Motivate Yourself: Change Your Lif...
book_author                                               Steve Chandler
year_of_publication                                           1997-01-01
Publisher                                               Highbridge Audio
Category                                                       self help
Category_other                                                       NaN
book_description                                                     NaN
book_language                                                         en
Image                                                                NaN
number_of_pages                                                       90
author_genres                                      

In [49]:
df_duplicates.head(1)

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,awards
0,345465083,"9,78E+12",0.0,Seabiscuit : An American Legend,Laura Hillenbrand,2003-01-01,"Random House Digital, Inc.",histori,NaN,"Retraces the journey of Seabiscuit, a horse wi...",en,http://books.google.com/books/content?id=kAvhr...,453.0,histori,NaN,[],National Book Critics Circle Award Nominee for...


In [46]:
print(len(books_scrapped_cleaned))
books_scrapped_cleaned.head(1)

99542


,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,awards
0,195153448,"9,78E+12",0.0,Classical Mythology,Mark P. O. Morford,2003-01-01,"Oxford University Press, USA",social sci,NaN,Provides an introduction to classical myths pl...,en,http://books.google.com/books/content?id=YABYK...,808.0,NaN,NaN,NaN,NaN


In [47]:
books_scrapped_cleaned.to_csv(file_scrapper_cleaned, sep = "|", index = False)

In [50]:
df_duplicates.to_csv(path_scrapper + "Books_Duplicates.csv", sep = "|", index = False)

# Category and author_genres selection

##### Join Category and author_genre of books_scrapped_cleaned and df_duplicates



In [6]:
books_scrapped_cleaned = pd.read_csv(file_scrapper_cleaned, sep = '|')
df_duplicates = pd.read_csv(path_scrapper + "Books_Duplicates.csv", sep = '|')
print(len(books_scrapped_cleaned))
books_scrapped_cleaned.head(1)

99542


,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,awards
0,195153448,"9,78E+12",0.0,Classical Mythology,Mark P. O. Morford,2003-01-01,"Oxford University Press, USA",social sci,NaN,Provides an introduction to classical myths pl...,en,http://books.google.com/books/content?id=YABYK...,808.0,NaN,NaN,NaN,NaN


In [7]:
#indexes are the categories names !
stats_cate = books_scrapped_cleaned["Category"].value_counts(sort = True)
cate = stats_cate.index.unique()

stats_aut_genr = books_scrapped_cleaned["author_genres"].value_counts(sort = True)
aut_genr = stats_aut_genr.index.unique()

In [9]:
complete_tab = stats_cate

for g in aut_genr:
    if g in cate:
        complete_tab[g] = stats_aut_genr[g] + stats_cate[g]
    else:
        complete_tab[g] = stats_aut_genr[g]

In [10]:
complete_tab.shape

(4366,)

In [11]:
stats_cate = df_duplicates["Category"].value_counts(sort = True)
cate = stats_cate.index.unique()

stats_aut_genr = df_duplicates["author_genres"].value_counts(sort = True)
aut_genr = stats_aut_genr.index.unique()

In [12]:
for g in aut_genr:
    if g in complete_tab:
        complete_tab[g] = complete_tab[g] + stats_aut_genr[g]
    else:
        complete_tab[g] = stats_aut_genr[g]

for g in cate:
    if g in complete_tab:
        complete_tab[g] = complete_tab[g] + stats_cate[g]
    else:
        complete_tab[g] = stats_cate[g]        

In [13]:
complete_tab.shape

(4587,)

In [50]:
complete_tab.sort_values(ascending = False)

fiction                        45933
mysteri                        12284
literatur                      11773
romanc                          9454
science fict                    8089
                               ...  
cambio                             1
social work administration.        1
nemška književnost - romani        1
shop                               1
roma                               1
Name: Category, Length: 4587, dtype: int64

In [ ]:
nb_cat_sel = 403
complete_tab.sort_values(ascending = False)[0:nb_cat_sel].to_csv(path_scrapper + "Sel_cat_autgenr.csv", sep = "|")
complete_tab.sort_values(ascending = False)[nb_cat_sel:complete_tab.shape[0]].to_csv(path_scrapper + "Other_cat_autgenr.csv", sep = "|")

##### Creation of train data set

In [15]:
books_scrapped_cleaned = pd.read_csv(file_scrapper_cleaned, sep = '|')
df_duplicates = pd.read_csv(path_scrapper + "Books_Duplicates.csv", sep = '|')
books_scrapped_cleaned = pd.concat([books_scrapped_cleaned, df_duplicates], ignore_index=True, axis = 0)
len(books_scrapped_cleaned)

112833

In [16]:
print(len(books_scrapped_cleaned))
books_scrapped_cleaned.head()

112833


,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,awards
0,195153448,"9,78E+12",0.0,Classical Mythology,Mark P. O. Morford,2003-01-01,"Oxford University Press, USA",social sci,NaN,Provides an introduction to classical myths pl...,en,http://books.google.com/books/content?id=YABYK...,808.0,NaN,NaN,NaN,NaN
1,2005018,"9,78E+12",0.0,Clara Callan : A Novel,Richard Bruce Wright,2001-01-01,HarperFlamingo Canada,actress,NaN,"In a small town in Canada, Clara Callan reluct...",en,http://books.google.com/books/content?id=yfx0v...,414.0,NaN,NaN,[],Trillium Book Award
2,60973129,0,IND:30000026059836,Decision in Normandy,Carlo D'Este,1991-01-01,Harper Perennial,1940-1949,NaN,"Here, for the first time in paperback, is an o...",en,http://books.google.com/books/content?id=_LufA...,555.0,histori,NaN,[],NaN
3,374157065,"9,78E+12",0,Flu : The Story of the Great Influenza Pandemi...,Gina Bari Kolata,1999-01-01,Macmillan,medic,NaN,"Describes the great flu epidemic of 1918, an o...",en,http://books.google.com/books/content?id=GkthX...,330.0,health,NaN,[],NaN
4,393045218,"9,78E+12",0.0,The Mummies of Ürümchi,E. J. W. Barber,1999-01-01,W. W. Norton,design,NaN,A look at the incredibly well-preserved ancien...,en,http://books.google.com/books/content?id=5OujQ...,240.0,histori,NaN,[],NaN


In [17]:
#We keep only few columns to create the context of training
df1 = books_scrapped_cleaned[["book_description", "Category", "Category_other", "author_genres", "author_genres_other"]]
df1.head()

,book_description,Category,Category_other,author_genres,author_genres_other
0,Provides an introduction to classical myths pl...,social sci,NaN,NaN,NaN
1,"In a small town in Canada, Clara Callan reluct...",actress,NaN,NaN,NaN
2,"Here, for the first time in paperback, is an o...",1940-1949,NaN,histori,NaN
3,"Describes the great flu epidemic of 1918, an o...",medic,NaN,health,NaN
4,A look at the incredibly well-preserved ancien...,design,NaN,histori,NaN


In [18]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
stopwords_list = stopwords.words('english')

In [20]:
stopwords_list[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [21]:
#with data set of about 100 000 lines, it takes 1 min
lines_OK = df1.book_description[df1.book_description.isna() == False]

for i in lines_OK.index:
    #we get sentence inside book_description column, we remove final '.' and separate all words
    w = df1.book_description[i].strip('.').split()

    #we remove stopwords, words of length < 2 and '...'
    ww = [word.strip(',') for word in w if (word not in stopwords_list) and len(word) > 2 and (word != '...')]

    #We reconstruct a sentence with the remaining words, separated with ','
    df1.book_description[i] = ','.join(ww)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [22]:
df1.book_description.head()

0    Provides,introduction,classical,myths,placing,...
1    small,town,Canada,Clara,Callan,reluctantly,tak...
2    Here,first,time,paperback,outstanding,military...
3    Describes,great,flu,epidemic,1918,outbreak,kil...
4    look,incredibly,well-preserved,ancient,mummies...
Name: book_description, dtype: object

In [23]:
#All those columns informations must be joined inside one column
#str(x) != "nan":     but we join only if column word is not empty
#x.split():           sentences in colum 'book_description' are cut in words
#','.join(x.split()): all those words are joined with (',') to be transformed after in list of list
def WordToJoin(w):
  ww = [x for x in w if str(x) != "nan"]
  return ','.join(ww)

df2 = df1.apply(lambda x: WordToJoin(x), axis = 1)
df2.head()

0    Provides,introduction,classical,myths,placing,...
1    small,town,Canada,Clara,Callan,reluctantly,tak...
2    Here,first,time,paperback,outstanding,military...
3    Describes,great,flu,epidemic,1918,outbreak,kil...
4    look,incredibly,well-preserved,ancient,mummies...
dtype: object

In [24]:
#a specific data frame is created with all this train context
df_clean = pd.DataFrame({'clean': df2})
df_clean.head()

,clean
0,"Provides,introduction,classical,myths,placing,..."
1,"small,town,Canada,Clara,Callan,reluctantly,tak..."
2,"Here,first,time,paperback,outstanding,military..."
3,"Describes,great,flu,epidemic,1918,outbreak,kil..."
4,"look,incredibly,well-preserved,ancient,mummies..."


In [25]:
sent = [row.split(',') for row in df_clean['clean']]
sent[0]

['Provides',
 'introduction',
 'classical',
 'myths',
 'placing',
 'addressed',
 'topics',
 'within',
 'historical',
 'context',
 'discussion',
 'archaeological',
 'evidence',
 'support',
 'mythical',
 'events',
 'themes',
 'portrayed',
 'literature',
 'art',
 'social sci']

##### Skip Gram model training

In [26]:
!pip install --upgrade gensim

     |████████████████████████████████| 23.9MB 1.3MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [27]:
from gensim.models import Word2Vec

/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [28]:
#sg: 0 for CBOW / 1 for skip gram
#window: distance between target word and around words inside context
model = Word2Vec(min_count = 1, vector_size = 50, workers = 3, window = 3, sg = 1)

In [30]:
type(model)

gensim.models.word2vec.Word2Vec

In [31]:
model.build_vocab(sent)

In [32]:
model.train(sent, total_examples = model.corpus_count, epochs = model.epochs)

(6891223, 7330380)

In [33]:
for i in range(2):
  w = books_scrapped_cleaned.Category[i]
  print("Mot original: %s\n" % w)
  print(model.wv[w], "\n\n")
  print(model.wv.most_similar(w, topn=10), "\n\n")

Mot original: social sci

[ 0.9119811  -1.5320835   1.062959    0.87213725 -1.2876016  -0.18897228
  1.1404556   0.13631254 -0.639257    0.27337053 -0.5757122  -0.13806535
  0.47136414  0.09954336 -0.8325257  -0.22439162 -0.3548262  -0.7447521
 -0.68179804 -0.03864497 -0.54879487  0.68530333  1.3502498   0.5358151
 -0.09780243  0.22972333 -0.69660836 -0.3257176  -0.44608447 -0.19397044
  0.17624897  0.01115282  0.30963677 -0.10906821  0.28507406  0.25516146
 -0.71367556 -0.01058627 -0.303773   -0.23146151  0.24702457  0.14805904
 -0.07033689  0.6294008   0.69545573 -0.7897713  -0.48992267  0.2974441
 -0.20150664  0.32073718] 


[('political sci', 0.9123600721359253), ('femin', 0.9025653004646301), ('literary collect', 0.8616941571235657), ('literary critic', 0.8616915345191956), ('psycholog', 0.8612384796142578), ('presid', 0.8598936796188354), ('american literatur', 0.854351818561554), ('refer', 0.8531622290611267), ('architectur', 0.8482690453529358), ('true crim', 0.8466387987136841

In [34]:
model.save(path_scrapper + "skipGram_cat.model")

##### Find the Category the most close to the selected categories

In [35]:
sel_cat = pd.read_csv(path_scrapper + "Sel_cat_autgenr.csv", sep = "|")
sel_cat = sel_cat.rename( columns = {"Unnamed: 0" :"Categories", "Category": "nb_Books_Inside"} )

other_cat = pd.read_csv(path_scrapper + "Other_cat_autgenr.csv", sep = "|")
other_cat = other_cat.rename( columns = {"Unnamed: 0" :"Categories", "Category": "nb_Books_Inside"} )

In [36]:
other_cat.head()

,Categories,nb_Books_Inside,Category_Close
0,dent,9,baggin
1,film novel,9,life on other planet
2,caricatures and cartoon,9,sequential art
3,spy thril,9,intelligence servic
4,good and evil,9,vampir


In [ ]:
sel_cat.head()

,Categories,nb_Books_Inside
0,fiction,46231
1,mysteri,12402
2,literatur,11902
3,romanc,9479
4,science fict,8161


In [37]:
sel_cat_list = list(sel_cat.Categories)

In [ ]:
print(len(sel_cat_list))
sel_cat_list[0:5]

403


['fiction', 'mysteri', 'literatur', 'romanc', 'science fict']

In [ ]:
other_cat.Categories[1]

'film novel'

In [ ]:
model.wv.most_similar_to_given(key1 = other_cat.Categories[1], keys_list = sel_cat_list)

'fantasy gam'

In [38]:
for i in range(other_cat.shape[0]):
  other_cat["Category_Close"][i] = model.wv.most_similar_to_given(key1 = other_cat.Categories[i], keys_list = sel_cat_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [39]:
other_cat.head()

,Categories,nb_Books_Inside,Category_Close
0,dent,9,baggin
1,film novel,9,fantasy gam
2,caricatures and cartoon,9,sequential art
3,spy thril,9,intelligence servic
4,good and evil,9,dystopia


In [ ]:
other_cat.to_csv(path_scrapper + "Other_cat_autgenr.csv", sep = "|", index = False)

##### Cleaning of Category and Author_genre

In [40]:
books_scrapped_cleaned = pd.read_csv(file_scrapper_cleaned, sep = '|')

sel_cat = pd.read_csv(path_scrapper + "Sel_cat_autgenr.csv", sep = "|")
sel_cat = sel_cat.rename( columns = {"Unnamed: 0" :"Categories", "Category": "nb_Books_Inside"} )
sel_cat_list = list(sel_cat.Categories)

other_cat = pd.read_csv(path_scrapper + "Other_cat_autgenr.csv", sep = "|")
other_cat = other_cat.rename( columns = {"Unnamed: 0" :"Categories", "Category": "nb_Books_Inside"} )

In [41]:
sel_cat.head()

,Categories,nb_Books_Inside
0,fiction,46231
1,mysteri,12402
2,literatur,11902
3,romanc,9479
4,science fict,8161


In [42]:
other_cat.head()

,Categories,nb_Books_Inside,Category_Close
0,dent,9,baggin
1,film novel,9,life on other planet
2,caricatures and cartoon,9,sequential art
3,spy thril,9,intelligence servic
4,good and evil,9,vampir


In [43]:
books_scrapped_cleaned.head(5)

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,awards
0,195153448,"9,78E+12",0.0,Classical Mythology,Mark P. O. Morford,2003-01-01,"Oxford University Press, USA",social sci,NaN,Provides an introduction to classical myths pl...,en,http://books.google.com/books/content?id=YABYK...,808.0,NaN,NaN,NaN,NaN
1,2005018,"9,78E+12",0.0,Clara Callan : A Novel,Richard Bruce Wright,2001-01-01,HarperFlamingo Canada,actress,NaN,"In a small town in Canada, Clara Callan reluct...",en,http://books.google.com/books/content?id=yfx0v...,414.0,NaN,NaN,[],Trillium Book Award
2,60973129,0,IND:30000026059836,Decision in Normandy,Carlo D'Este,1991-01-01,Harper Perennial,1940-1949,NaN,"Here, for the first time in paperback, is an o...",en,http://books.google.com/books/content?id=_LufA...,555.0,histori,NaN,[],NaN
3,374157065,"9,78E+12",0,Flu : The Story of the Great Influenza Pandemi...,Gina Bari Kolata,1999-01-01,Macmillan,medic,NaN,"Describes the great flu epidemic of 1918, an o...",en,http://books.google.com/books/content?id=GkthX...,330.0,health,NaN,[],NaN
4,393045218,"9,78E+12",0.0,The Mummies of Ürümchi,E. J. W. Barber,1999-01-01,W. W. Norton,design,NaN,A look at the incredibly well-preserved ancien...,en,http://books.google.com/books/content?id=5OujQ...,240.0,histori,NaN,[],NaN


In [44]:
books_scrapped_cleaned["Category"].head()

0    social sci
1       actress
2     1940-1949
3         medic
4        design
Name: Category, dtype: object

In [45]:
lines_OK = books_scrapped_cleaned.Category[books_scrapped_cleaned["Category"].isna() == False]

for i in lines_OK.index:
  #if categories in not insed the set selected
  if (books_scrapped_cleaned.Category[i] not in sel_cat_list):
    w = books_scrapped_cleaned.Category[i]

    #Then we search inside 'other_cat' the closest categorie found thanks to word2vec
    w_close = other_cat[other_cat.Categories == w]["Category_Close"]
    books_scrapped_cleaned.Category[i] = w_close.values[0]
    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [46]:
lines_OK = books_scrapped_cleaned.author_genres[books_scrapped_cleaned["author_genres"].isna() == False]

for i in lines_OK.index:
  if (books_scrapped_cleaned.author_genres[i] not in sel_cat_list):
    w = books_scrapped_cleaned.author_genres[i]
    w_close = other_cat[other_cat.Categories == w]["Category_Close"]
    books_scrapped_cleaned.author_genres[i] = w_close.values[0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [47]:
books_scrapped_cleaned.head()

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,awards
0,195153448,"9,78E+12",0.0,Classical Mythology,Mark P. O. Morford,2003-01-01,"Oxford University Press, USA",social sci,NaN,Provides an introduction to classical myths pl...,en,http://books.google.com/books/content?id=YABYK...,808.0,NaN,NaN,NaN,NaN
1,2005018,"9,78E+12",0.0,Clara Callan : A Novel,Richard Bruce Wright,2001-01-01,HarperFlamingo Canada,actress,NaN,"In a small town in Canada, Clara Callan reluct...",en,http://books.google.com/books/content?id=yfx0v...,414.0,NaN,NaN,[],Trillium Book Award
2,60973129,0,IND:30000026059836,Decision in Normandy,Carlo D'Este,1991-01-01,Harper Perennial,spanish languag,NaN,"Here, for the first time in paperback, is an o...",en,http://books.google.com/books/content?id=_LufA...,555.0,histori,NaN,[],NaN
3,374157065,"9,78E+12",0,Flu : The Story of the Great Influenza Pandemi...,Gina Bari Kolata,1999-01-01,Macmillan,medic,NaN,"Describes the great flu epidemic of 1918, an o...",en,http://books.google.com/books/content?id=GkthX...,330.0,health,NaN,[],NaN
4,393045218,"9,78E+12",0.0,The Mummies of Ürümchi,E. J. W. Barber,1999-01-01,W. W. Norton,design,NaN,A look at the incredibly well-preserved ancien...,en,http://books.google.com/books/content?id=5OujQ...,240.0,histori,NaN,[],NaN


In [48]:
books_scrapped_cleaned.Category.value_counts()

fiction           34119
juvenile fict      5194
biographi          2958
mysteri            2241
histori            1883
                  ...  
film                  2
chick lit             2
theatr                2
profession            1
historical rom        1
Name: Category, Length: 396, dtype: int64

In [49]:
books_scrapped_cleaned.to_csv(file_scrapper_w2v, sep = "|", index = False)